## Vue with DRF

### server & client
- 서비스 전체를 제공 == Django Web Service
    - Django가 모든 내용을 포함한 HTML을 전달
    - 모든 내용을 server 가 제공(새로고침)
- 정보를 제공 == DRF API Service
    - Django를 통해 관리하는 정보만을 클라이언트에게 제공
    - DRF를 사용하여 JSON을 변환
- 클라이언트란?
    - server가 제공하는 서비스에 적절한 요청을 토해 server로부터 반환 받은 응답을 사용자에게 표현하는 기능을 가진 프로그램 혹은 시스템

### CORS

#### Cross-Origin Resource Sharing
- 브라우저가 요청을 보내고 서버의 응답이 브라우저에 도착
    - server의 log는 200 반환
    - 즉 서버는 정상적으로 응답했지만 브라우저가 막은것
- 보안상의 이유로 브라우저는 동일 출처 정책(SOP)에 의해 다른 출처의 리소스와 상호작용 하는 것을 제한 함

##### SOP(Same-Origin Policy)
- 동일 출처 정책
- 불러온 문서나 스크립트가 다른 출처에서 가져온 리소스와 상호작용 하는 것을 제한하는 보안 방식
- 잠재적으로 해로울 수 있는 문서를 분리함으로써 공격받을 수 있는 경로를 줄임
- URL의 Protocol, Host, Port를 모두 포함하여 출처라고 부름
- htpp://localhost:3000/posts/3 여기서 http(프로토콜)부터 3000(포트)까지 같아야 출처가 같은것

#### CORS - 교차출처 리소스 공유
- 추가 HTTP Header를 사용하여 특정 출처에서 실행중인 웹 어플리케이션이 다른 출처의 자원에 접근할 수 있는 권한을 부여하도록 브라우저에 알려주는 체제
    - 어떤 출처에서 자신의 컨텐츠를 불러갈 수 있는지 서버에 지정할 수 있는 방법
- 리소스가 자신의 출처와 다를 때 교차 출처 HTTP 요청을 실행
    - 만약 다른 출처의 리소스를 가져오기 위해서는 이를 제공하는 서버가 브라우저에게 다른 출처지만 접근해도 된다는 사실을 알려야 함
    - 교차 출처 리소스 공유 정책(CORS policy)
- CORS policy에 위배되는 경우 브라우저에서 해당 응답 결과를 사용하지 않음
    - server에서 응답을 주더라도 브라우저에서 거절
- 다른 출처의 리소스 불러올시 그 출처에서 올바른 CORS header를 포함한 응답을 반환해야 함

### How to set CORS
- CORS 표준에 의해 추가된 HTTP Response Header를 통해 이를 통제 가능
    - 예시: Access-Control-Allow-Origin/ ...-Credentials/ ...-Headers/ ...-Methods
- ...Origin: 단일 출처를 지정하여 브라우저가 해당 출처가 리소스에 접근하도록 허용
- django-cors-headers library 사용
    - 응답에 CORS header를 추가해주는 라이브러리
    - 다른 출처에서 Django 애플리케이션에 대한 브라우저 내 요청을 허용함
    - pip install django-cors-headers
    - settings.py에서 app에 'corsheaders' 추가 및 MIDDLEWARE에 'corsheaders.middleware.CorsMiddleware', 'django.middleware.common.CommonMiddleware' 추가(순서는 꼭 이대로)
    - CORS_ALLOWED_ORIGINS = [ 'http://localhost:8080', ] : 교차 출처 자원 공유를 허용할 Domain 등록, 모든 Origin 허용시 = True

## Authentication & Authorization

### Authentication - 입증, 인증
- 자신이라고 주장하는 사용자가 누구인지 확인하는 행위
- 모든 보안 프로세스의 첫 번째 단계(가장 기본 요소)
- 401 Unauthorized
    - HTTP 표준에서는 미승인(unauthorized)을 명확히 하고있지만 의미상 이 응답은 비인증(unauthenticated)을 의미

### Authorization
- 사용자에게 특정 리소스 또는 기능에 대한 액세스 권한을 부여하는 과정(절차)
- 보안 환경에서 권한 부여는 항상 인증이 먼저 필요함
    - 사용자는 조직에 대한 액세스 권한을 부여받기 전에 먼저 자신의 ID가 진짜인지 먼저 확인해야 함
- 서류의 등급, 웹 페이지에서 글을 조회 & 삭제 & 수정 할 수 있는 방법, 제한 구역
    - 인증이 되었어도 모든 권한을 부여 받는 것은 아님
- 403 Forbidden
    - 401과 다른 점은 서버는 클라이언트가 누구인지 알고 있음

### 인증 여부 확인 방법
- settings.py에 작성하여야 할 설정
    - 기본적인 인증 절차를 어떠한 방식으로 둘 것이냐를 설정하는 것
우리가 사용할 방법은 DRF가 기본으로 제공해주는 인증 방식 중 하나인 TokenAuthentication
- 모든 상황에 대한 인증 방식을 정의 하는 것이므로 각 요청에 따라 다른 인증 방식을 거치고자 한다면 다른 방식이 필요
- view 함수마다(각 요청마다) 다른 방식을 설정하고자 한다면 decorator 활용
- [참고] permission_classes
    - 권한 관련 설정
    - 권한 역시 특정 view 함수마다 다른 접근 권한을 요구할 수 있음

### 다양한 인증 방식
- BasicAuthentication
    - 가장 기본적인 수준의 인증 방식
    - 테스트에 적합
- SessionAuthentication
    - Django에서 사용하였던 session 기반의 인증 시스템
    - DRF와 Django의 session 인증 방식은 보안적 측면을 구성하는 방법에 차이가 있음
- RemoteUserAuthentication
    - Django의 Remote user 방식을 사용할 때 활용하는 인증 방식
- TokenAuthentication
    - 매우 간단하게 구현 가능
    - 기본적인 보안 기능 제공
    - 다양한 외부 패키지 있음
- [중요] setting.py에서 DEFAULT_AUTHENTICATION_CLASSES를 정의
    - TokenAuthentication 인증 방식을 사용할 것임을 명시

#### TokenAuthentication 사용 방법
- setting.py INSTALLED_APPS에 rest_framework.authtoken 등록
- 각 User마다 고유 Token 생성
- 생성한 Token을 각 User에게 발급
    - User는 발급 받은 Token을 요청과 함께 전송
    - Token을 통해 User 인증 및 권한 확인
- User는 발급받은 Token을 headers에 담아 요청과 함께 전송
    - 단, 반드시 Token 문자열 함께 삽입
    - 삼입해야 할 문자열은 각 인증 방식마다 다름
    - Token 문자열과 발급받은 실제 token 사이를 ' '(공백)으로 구분

### dj-rest-auth
- dj-rest-auth 라이브러리 사용
- 패키지 설치 pip install dj-rest-auth
- 앱 등록: 'rest_framework', 'rest_framework.authtoken', 'dj_rest_auth'
- url 등록 : path('', include('dj_rest_auth.urls'))
- 회원가입 기능을 위해: pip install 'dj-rest-auth[with_social]'
- app 추가 : 'django.contrib.sites', 'allauth', 'allauth.account', 'allauth.socialaccount', 'dj-rest-auth.registration',
- SITE_ID = 1
    - Django는 하나의 컨텐츠를 기반으로 여러 도메인에 컨텐츠를 게시 가능하도록 설계됨
    - 다수의 도메인이 하나의 데이터베이스에 등록
    - 현재 프로젝트가 첫번째 사이트임을 나타냄
- path('', include('dj_rest_auth.registration.urls'))
- 인증 방법을 등록: REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.TokenAuthentication',
    ],
}

## permission setting
- 권한 세부 설정
    1. 모든 요청에 대해 인증을 요구하는 설정
    2. 모든 요청에 대해 인증 없이도 허용하는 설정
- 설정 위치 == 인증 방법을 설정한 곳과 동일
    - 우선 모든 요청에 대해 허용 설정